In [1]:
%%spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1628715867829_0008,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date,TimestampType as Tst


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
songSchema = R([
    Fld("num_songs",Int()),
    Fld("artist_id",Str()),
    Fld("artist_latitude",Dbl()),
    Fld("artist_longitude",Str()),
    Fld("artist_location",Str()),
    Fld("artist_name",Date()),
    Fld("song_id",Str()),
    Fld("title",Str()),
    Fld("duration",Dbl()),
    Fld("year",Int()),
    
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df = spark.read.json("s3a://udacity-dend/song_data/*/*/*/*.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [6]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|AR4T2IF1187B9ADBB7|       63.96027|<a href="http://b...|        10.22442|          Billy Idol|233.22077|        1|SOVIYJY12AF72A4B00|The Dead Next Doo...|1983|
|AR4T2IF1187B9ADBB7|       63.96027|<a href="http://b...|        10.22442|          Billy Idol|287.92118|        1|SOVYXYL12AF72A3373|Rebel Yell (1999 ...|1983|
|ARQ846I1187B9A7083|           null|                    |            null|Yvonne S. Moriart...|196.04853|        1|SOEPTVC12A67ADD0DA|To Zucchabar ["Gl...|   0|
|AR4T2IF1187B9ADBB7|       63.9602

In [7]:
df.createOrReplaceTempView("songs")
songs_table = spark.sql("""
    SELECT DISTINCT song_id, title, artist_id, year, duration
    FROM songs
    WHERE song_id IS NOT NULL
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
songs_table.write.mode("overwrite").partitionBy("year","artist_id").parquet("s3a://mynewlakebucket/songs/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df3 = spark.sql("""
    SELECT 
        DISTINCT artist_id, artist_name AS name, artist_location AS location, artist_latitude AS latitude , artist_longitude AS longitude 
    FROM songs
    WHERE artist_id IS NOT NULL
""")

In [ ]:
df3.show(5)

In [ ]:
df3.write.mode("overwrite").parquet("s3a://mynewlakebucket/artists/")

In [ ]:
df_log = spark.read.json("s3a://udacity-dend/log_data/*/*/*.json")

In [ ]:
df_log.printSchema()

In [ ]:
df_log = df_log.filter(df_log.page == 'NextSong')

In [ ]:
df_log.createOrReplaceTempView("log")

In [ ]:
df_users = spark.sql("""
    SELECT DISTINCT userId as user_id, firstName as first_name, lastName as last_name, gender, level
    FROM log
""")

In [ ]:
df_users.show(5)

In [ ]:
df_users.write.mode("overwrite").parquet("s3a://mynewlakebucket/users")

In [ ]:
import  pyspark.sql.functions as F
import datetime

In [ ]:
df_log_ts = df_log.select('ts')

In [ ]:
df_log_ts.show(1)

In [ ]:
df_log_ts.show(1)


In [ ]:
from pyspark.sql.functions import udf,col


In [ ]:
get_timestamp = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000),Tst())

In [ ]:
df_log = df_log.withColumn("datetime", get_timestamp(df_log.ts))

In [ ]:
df_log.select('datetime').show(5)

In [ ]:
df_log.createOrReplaceTempView("timetable")

In [ ]:
df_time = spark.sql("""
    SELECT 
        DISTINCT datetime as start_time, 
        EXTRACT(HOUR FROM datetime) as hour,
        EXTRACT(DAY FROM datetime) as day,
        EXTRACT(WEEK FROM datetime) as week,
        EXTRACT(MONTH FROM datetime) as month,
        EXTRACT(YEAR FROM datetime) as year,
        EXTRACT(DAYOFWEEK FROM datetime) as weekday
    FROM timetable

""")

In [ ]:
df_time.show(5)

In [ ]:
df_time.write.mode("overwrite").partitionBy("year","month").parquet("s3a://mynewlakebucket/time")

In [ ]:
df.createOrReplaceTempView("songs")


In [ ]:
df_log.printSchema()
df.printSchema()

In [ ]:
df_songplays = spark.sql("""
    SELECT
        
        t.datetime as start_time,
        t.userId, t.level, s.song_id, s.artist_id,
        t.sessionId, s.artist_location, t.userAgent,
        EXTRACT(MONTH FROM t.datetime) as month,
        EXTRACT(YEAR FROM t.datetime) as year
    FROM songs s,timetable t
    WHERE t.artist = s.artist_name
    AND t.song = s.title
""")

In [ ]:
df_songplays.show(5)

In [ ]:
import pyspark.sql.functions as psf

In [ ]:
df_songplays = df_songplays.withColumn('songplay_id', psf.monotonically_increasing_id())


In [ ]:
df_songplays.write.mode("overwrite").partitionBy("year","month").parquet("s3a://mynewlakebucket/" + "songplays/")